In [1]:
import numpy as np 

from MissingImputer import MissingImputer

import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
from utils import load_train_test,make_plot,RMSE,mask_check
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]
#real_datalist = ["yacht_hydrodynamics"]

syn_datalist = ["syn1"]


missingtypelist = ["quantile","diffuse","logistic"]
#missingtypelist = ["diffuse"]

In [3]:
def imputer_model(missingtypelist,datalist_name,model_name,seed = 1,nfold = 5):

    

    if datalist_name == "real":
        datalist = real_datalist
    elif datalist_name == "syn":
        datalist = syn_datalist

    for missingtype in missingtypelist:
        print(missingtype)
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")



        for dataname in datalist:
            print(dataname)
            path = f"../impute/{missingtype}/{dataname}/{model_name}"
            if not os.path.exists(path):
                # If the path does not exist, create it
                os.makedirs(path)

            for rule_name in missing_rule:
                    print(rule_name)

                    directory_path = f"../datasets/{dataname}"  
                    # Opening JSON file
                    f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
                    index_file = json.load(f)

                    for fold in index_file:
                        print(fold)
                        train_values,train_masks,test_values,test_masks = load_train_test(index_file[fold],missingtype,rule_name,directory_path,dataname)

                        train_values_na = np.where(train_masks == 0, np.nan, train_values)
                        test_values_na = np.where(test_masks == 0, np.nan, test_values)
                        
                        imp = MissingImputer(ini_fill = True, model_reg = "xgboost", model_clf = "xgboost")
                        imp.fit(train_values_na,model_params = {'regressor':{'booster': 'gblinear'}, 'classifier':{'booster': 'gblinear'}})\
                        
                        test_imp = imp.transform(test_values_na)
                        train_imp = imp.transform(train_values_na)

                        print("tobe save")

                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_train.npy', train_imp.astype("float32"))
                        np.save(f'{path}/{rule_name}_seed-{seed}_{fold}_test.npy', test_imp.astype("float32"))

                        print("saved")
                        if  (test_imp.shape != test_values_na.shape):
                            print("test shape")
                        if  (train_imp.shape != train_values_na.shape):
                            print("train shape")


In [4]:
real_datalist = [
            #"banknote",
            #"concrete_compression",
            #  "wine_quality_white",
            # "wine_quality_red",
            # "california",
            # "climate_model_crashes",
            # "connectionist_bench_sonar",
            # "qsar_biodegradation", 
            # "yeast", 
            # "yacht_hydrodynamics"
            ]
missingtypelist = [
                    "quantile",
                   "diffuse",
                   "logistic"
                   ]

imputer_model(missingtypelist,"syn","XGB")

quantile
syn1
Q1_Q2_0.25
fold_1
[[0.52361063 0.36866484 0.57959604 ... 0.91253949 0.20736699 0.25270661]
 [0.11072814 0.57709088 0.5        ... 0.70802999 0.84669853 0.24634095]
 [0.46211591 0.31628645 0.75       ... 0.43964621 0.74559362 0.32433221]
 ...
 [0.31677877 0.51288158 0.25       ... 0.66089435 0.30779629 0.22037699]
 [0.7016512  0.46641022 1.         ... 0.83603361 0.33848261 0.31868857]
 [0.44578617 0.49598998 0.5        ... 0.9056575  0.78126336 0.21991289]]
tobe save
saved
fold_2
[[0.51478165 0.56221797 0.60042429 ... 0.73830169 0.24484679 0.41617353]
 [0.48620981 0.3483114  0.75       ... 0.70073218 0.52872378 0.89073481]
 [0.59220527 0.69146081 0.25       ... 0.63177048 0.81909872 0.31416634]
 ...
 [0.31677877 0.54300261 0.25       ... 0.66089435 0.30779629 0.22037699]
 [0.7016512  0.46641022 1.         ... 0.83603361 0.33848261 0.31702331]
 [0.44578617 0.53680176 0.5        ... 0.9056575  0.78126336 0.21991289]]
tobe save
saved
fold_3
[[0.51478165 0.56221797 0.59766513

e:\ANACONDA\envs\py3.10\lib\site-packages\xgboost\core.py:160: UserWarning: [19:45:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:52: Empty dataset at worker: 0
  warnings.warn(smsg, UserWarning)


[[0.52361063 0.36866484 0.25       ... 0.91253949 0.20736699 0.25270661]
 [0.11072814 0.57709088 0.5        ... 0.70802999 0.84669853 0.24634095]
 [0.40937708 0.31628645 0.75       ... 0.43964621 0.74559362 0.20897833]
 ...
 [0.31677877 0.31074584 0.25       ... 0.66089435 0.30779629 0.22037699]
 [0.7016512  0.46641022 1.         ... 0.83603361 0.33848261 0.21865003]
 [0.44578617 0.42230245 0.5        ... 0.9056575  0.78126336 0.21991289]]
tobe save
saved
fold_2
[[0.51478165 0.56221797 0.         ... 0.5258166  0.24484679 0.41617353]
 [0.48620981 0.3483114  0.75       ... 0.70073218 0.55913734 0.89073481]
 [0.59220527 0.69146081 0.25       ... 0.7499035  0.81909872 0.18218164]
 ...
 [0.31677877 0.31074584 0.25       ... 0.66089435 0.30779629 0.22037699]
 [0.7016512  0.46641022 1.         ... 0.83603361 0.33848261 0.21865003]
 [0.44578617 0.42230245 0.5        ... 0.9056575  0.78126336 0.21991289]]
tobe save
saved
fold_3
[[0.51478165 0.56221797 0.         ... 0.5258166  0.24484679 0.416